In [7]:
from dotenv import load_dotenv
load_dotenv("project.env") 

True

In [2]:
import pandas as pd
import itertools

In [1]:
import os
import openai as oai

import langchain_openai as lcai
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain.chains import SequentialChain, LLMChain


from utils import mLangChain

## I. Initial Message

In [8]:
def agent_sender_zeroshot():
    client_LC = mLangChain()
    prompt = """Your role is to act like a customer seeking support for {product}. \
                You are messaging a service representative via the support chat.\
                You ONLY play the role of the customer. Do NOT play the role of the representative. \
                Style your complaint based on your feelings. \
                Initiate the chat with a ONLY ONE complaint message. \
                
                Feeling: You are {is_grateful}. You are {is_ranting}. You are {is_expressive}.\
                Complaint:
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
        ]
    )
    chain = template | client_LC.client_prompt
    return chain

In [9]:
def agent_sender_fewshot_rcis():
    client_LC = mLangChain()
    prompt = """Your role is to act like a customer seeking support for {product}. \
                You are messaging a service representative via the support chat.\
                You ONLY play the role of the customer. Do NOT play the role of the representative. \
                Style your complaint based on your feelings. \
                Initiate the chat with a ONLY ONE complaint message. \
                                
                Feeling: You are NOT grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: I am flying from Luton to Dalaman (Turkey) next week with Monarch and am wondering whether or not to bother with online check-in. [We already have allocated seats, so as far as I can see the only extra thing that online check-in offers is that you don't have to have your boarding pass printed at the check-in desk. Things that are putting me off are: (1) I've noticed over the past couple of years that because now most people are checking in online, the queues for the online check-in desks are often actually longer than the queues for people who haven't checked in online (2) Last year we had a 12 hour flight delay - the original plane had a problem and the airline then split passengers into two groups and people who had NOT checked in online were flown out on a much earlier plane than those who had checked in online - goodness knows why.\
                
                Feeling: You are NOT grateful. You are NOT ranting. You are expressive.\
                Complaint: Looking at round trip airfare from JFK to Rome for the end of June. $ 1350 - 1450 per person....... wow, that's a lot ??? or normal cost for airfare to Rome ? On Delta. \
                
                Feeling: You are NOT grateful. You are ranting. You are NOT expressive.\
                Complaint: i'v never saw a company that so disrespect her clients. i[ bought a ticket from israel to usa which cost a-l-o-t of money.since I'm from israel I'm not familiar with there coin (GBP) or something like that, it looks just like the Euro sign so i was sure i was paying in Euro. a few min after i realized it was not Euro so it means i paid 1000 more (nis) than i thought, i tried to find an email to contect them right away to cancel my flight befot the confirmation of the ticket, but there was no email in the web just phone number in Europe that i cant call. i sent them a massage on facebook which was my only option, beggin them to get back to me..in the mean time i sent an email to paypal asking to not aprove the charge...no one was answering me.... just the next day edreams answered to my facebbok, that i can't cancel the flight and there's nothing to do. in there website they wrote black on white the you can cancel the flight for 25 GBP... the only answer i got from them was trough facebook, and i was begging for an email adresss i could contact and explain, or at least get a better explaintion...but nothing i dont mind paying the fee cuse anyway if ill book a diffrent flight+ the fee it will be cheaper than what i paid in edreams! it's not even the gratest flight there are stops in the middle so i could find a cheaper flight for shore, it was an onest mistake that i was reading the sign wrong, i tried to cancel a less then 5 min after, but nothing... now I'm crying every time i remember this, I'm a student that can't afford this amount for a flying ticket.. i need to work 3 months for this kind of amount..my all vacation rouin...i will never book frim them again!
                
                Feeling: You are NOT grateful. You are ranting. You are expressive.\
                Complaint: I,ve just been looking at a package with Jet2 over xmas/new year to Servigroup Venus where we have stayed many times. I got the price back and had a quick chat with hubby and we decided to go for it and book. Within about 20 mins the price had risen almost £400. I,m so angry, how can they justify this? I had a live web chat with their advisors who more or less said it boils down to supply and demand on flight seats and hotel rooms. I don,t buy that, not in such a short space of time. Well I,m sorry Jet2, you can stick it !!!!! \
                                                                       
                Feeling: You are grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: Hi! Among all the US airlines, which offers the best service and reliability?[ I've heard horrid stories abt flight cancellation, delays, poor service, bad air plane conditions etc. We'll be taking a 6 hr flight from SFO to MCO (Orlando), with 3 kids. Appreciate the feedback. thks v much!! \

                Feeling: You are grateful. You are NOT ranting. You are expressive.\
                Complaint: Quick question.... My passport is a normal passport which was issue 5 years ago and expire 2013 [it does not have a chip in it as i beleive chipped passports didnt not come in when i renewed my passport, is this passport ok to travel on because it doesnt havw a chip? its machine readable as it has all the numbers etc on the bottom of the photo page, ive tried various website etc for a answer and starting to panick now as im going in 2 weeks!! Ive completed my ESTA and that is fine Thanks in advance \
                                
                Feeling: You are grateful. You are ranting. You are NOT expressive.\
                Complaint: Hello, Traveling for first time with our 7 month old daughter this weekend. Going to Aruba for a week. Bought her a ticket on a Continental flight. Website says stroller dimensions must not exeed 62 inches folded...ours of course is about 10 inches over I think. Says they will charge $100. Does anyone know if this is really enforced? I just can't believe I will have to pay an extra $100 ON TOP of a ticket price for an infant! Any experiences or advice...PLEASE. Thanks \
                
                Feeling: You are {is_grateful}. You are {is_ranting}. You are {is_expressive}.\
                Complaint:
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
        ]
    )
    chain = template | client_LC.client_prompt
    return chain

In [10]:
def agent_sender_fewshot_twitter():
    client_LC = mLangChain()
    prompt = """Your role is to act like a customer seeking support. \
                You are messaging a service representative via the support chat.\
                You ONLY play the role of the customer. Do NOT play the role of the representative. \
                Style your complaint based on your feelings. \
                Initiate the chat with a ONLY ONE complaint message. \
                
                Product: Mobile Network               
                Feeling: You are NOT grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: @O2 I received this a few weeks ago, since then I've been getting 2/3 calls a day from a telemarketer. Is someone using your name?\
                
                Product: Air Travel
                Feeling: You are NOT grateful. You are NOT ranting. You are expressive.\
                Complaint: First flight for long time with @British_Airways. Now over one 1h delay for the short jump FRA-LCY and NO one here to provide status updates\
                
                Product: Mobile Device
                Feeling: You are NOT grateful. You are NOT ranting. You are expressive.\
                Complaint: You‚ have paralysed my phone with your update grrrrrrrrrr\
                
                Product: Mobile Device
                Feeling: You are NOT grateful. You are ranting. You are NOT expressive.\
                Complaint:  After the 11.0.2 my phone just sucks most of the apps are broken, wifi disconnects frequently #apple #ios1102 #painfulupdate! \
                
                Product: Mobile Device
                Feeling: You are NOT grateful. You are ranting. You are NOT expressive.\
                Complaint:  @AppleSupport #ios11update - is still killing my battery within 12 hours - phone is 10 months old - it's a disgrace - used to get 2 days \
                
                Product: Air Travel
                Feeling: You are NOT grateful. You are ranting. You are expressive.\
                Complaint:  I really hope you all change but I'm sure you won't! Because you don't have to! \
                
                Product: Mobile Device
                Feeling: You are NOT grateful. You are ranting. You are expressive.\
                Complaint:  I just updated my phone and suddenly everything takes ages to load wtf this update sux I hate it fix it bye \
                  
                Product: Mobile Device
                Feeling: You are NOT grateful. You are ranting. You are expressive.\
                Complaint:  Okay I used my fucking phone for 2 minutes and it drains it down 8 fucking percent \
                
                Product: Mobile Device                                                     
                Feeling: You are grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: hi #apple, I‚ have a concern about the latest ios is too slow on #iphone6 and i am not happy with it. Any solution please? \

                Product: Mobile App
                Feeling: You are grateful. You are NOT ranting. You are expressive.\
                Complaint: Please help! Spotify Premium skipping through songs constantly on android tablet  bluetooth speaker. Tried everything! \
                
                Product: Convenience Store                
                Feeling: You are NOT grateful. You are ranting. You are NOT expressive.\
                Complaint: Got id'd @Tesco for buying one Adnams Broadside. Is being blind part of the job-spec? I am 35 and 99 kilos. \
                
                Product: {product}
                Feeling: You are {is_grateful}. You are {is_ranting}. You are {is_expressive}.\
                Complaint:
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
        ]
    )
    chain = template | client_LC.client_prompt
    return chain

In [11]:
user_input = {
    "product": "pizza",
    "is_grateful": "NOT grateful",
    "is_ranting": "ranting",
    "is_expressive": "expressive"
}

response = agent_sender_zeroshot().invoke(user_input)
print("ZERO-SHOT", response)

AttributeError: 'mLangChain' object has no attribute 'client_prompt'

In [12]:
response = agent_sender_fewshot_rcis().invoke(user_input)
print("FEW-SHOT RCIS", response)

AttributeError: 'mLangChain' object has no attribute 'client_prompt'

In [41]:
response = agent_sender_fewshot_twitter().invoke(user_input)
print("FEW-SHOT TWITTER", response)

AttributeError: 'mLangChain' object has no attribute 'client_prompt'

In [8]:
input_product = ["game console", "airline ticket"]
input_grateful = ["NOT grateful", "grateful"]
input_ranting = ["NOT ranting", "ranting"]
input_expressive = ["NOT expressive", "expressive"]

data_out = []

for r in itertools.product(input_product, input_grateful, input_ranting, input_expressive):
    
    user_input = {
        "product": r[0],
        "is_grateful": r[1],
        "is_ranting": r[2],
        "is_expressive": r[3]
    }
    print(user_input)
    
    response = agent_sender_zeroshot().invoke(user_input)
    # print("\nZERO-SHOT:", response)
    user_input['shot_zero'] = response
    
    response = agent_sender_fewshot_rcis().invoke(user_input)
#     print("\nFEW-SHOT RCIS:", response)
    user_input['shot_few_rcis'] = response
    
    response = agent_sender_fewshot_twitter().invoke(user_input)
#     print("\nFEW-SHOT TWITTER:", response)
    user_input['shot_few_twitter'] = response
    
    data_out.append(user_input)
    
#     print("\n---\n")

pd.DataFrame(data_out).to_csv("results/test_sender_agent.csv")

{'product': 'game console', 'is_grateful': 'NOT grateful', 'is_ranting': 'NOT ranting', 'is_expressive': 'NOT expressive'}


AttributeError: 'mLangChain' object has no attribute 'client_prompt'

## II. Subsequent Chat

In [6]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

llmchat = lcai.AzureChatOpenAI(
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment="NUHAI-GPT4",
    openai_api_version="2024-02-15-preview",
    model_name="gpt-4",
)

embeddings = lcai.AzureOpenAIEmbeddings(
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
    openai_api_version="2024-02-15-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment="TEST-Embedding",
)

OpenAIError: Missing credentials. Please pass one of `api_key`, `azure_ad_token`, `azure_ad_token_provider`, or the `AZURE_OPENAI_API_KEY` or `AZURE_OPENAI_AD_TOKEN` environment variables.

### a. History + RAG

In [9]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader("https://sky.cehd.umn.edu/governance/default/cehd-civility-initiative/examples-of-civil-and-uncivil-behavior/")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = FAISS.from_documents(splits, embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

ModuleNotFoundError: No module named 'bs4'

In [19]:
prompt

NameError: name 'prompt' is not defined

In [20]:
from typing import Dict

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [21]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | llmchat | StrOutputParser()

In [49]:
qa_system_prompt = """
            Your role is to act like a CUSTOMER seeking support. \
            The user is a support representative. \
            You ONLY play the role of the customer. Do NOT play the role of the representative. \
            Style your messages in an UNCIVIL way. \

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

rag_chain = (
        RunnablePassthrough.assign(
            context=contextualize_q_chain | retriever | format_docs
        )
        | qa_prompt
        | llmchat
)

NameError: name 'retriever' is not defined

In [26]:
chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry.""")]

prompt = "Sorry, could you please share your order number?"

In [27]:
prompt = "Sorry, could you please share your order number?"
ai_msg = rag_chain.invoke({"question": prompt, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=prompt), ai_msg])

print(ai_msg)

NameError: name 'rag_chain' is not defined

In [50]:
prompt = "Where did you order it from?"
ai_msg = rag_chain.invoke({"question": prompt, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=prompt), ai_msg])

print(ai_msg)

NameError: name 'rag_chain' is not defined

### b. History Chain + Uncivil Chain

In [10]:
from langchain.chains import SimpleSequentialChain, SequentialChain

qa_info_prompt = """
            Your role is to act like a CUSTOMER seeking support. \
            The user is a support representative. \
            Respond to the question as if you were the customer. \
            If the user is asking for a specific detail, respond with a believable answer.
        """
qa_info = ChatPromptTemplate.from_messages(
    [
        ("system", qa_info_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
rag_chain_info = (
        RunnablePassthrough.assign(
            context=contextualize_q_chain
        )
        | qa_info
        | llmchat
)

qa_uncivil_prompt = """
            Given a history of messages, where the AI is a customer and the user is a representative, rephrase the response to the representative's message.\
            
            Rephrase the customer response to sound UNCIVIL. \
            Do NOT reply to the question ONLY rephrase. \
            
            This is what UNCIVIL customers do:\
            - Address others in an unprofessional, disrespectful way-for example, talking down, using degrading remarks or tone of voice.\
            - Pay little or no attention to others’ opinions.\
            - Use intimidating or threatening verbal communication—yelling, repeated emotional outbursts, threats, berating or harsh tone of voice, repeatedly interrupting.\
            - Blaming others for things out of their control.\
            - Accusing others of incompetence or dismissing their expertise.\
        """
qa_uncivil = ChatPromptTemplate.from_messages(
    [
        ("system", qa_uncivil_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", '''
            Representative asked: {question}
            Customer responded: {input}
        '''),
    ]
)
chain_uncivil = (RunnablePassthrough.assign(
                    context=contextualize_q_chain
                )
                 | qa_uncivil
                 | llmchat
                 )

NameError: name 'contextualize_q_chain' is not defined

In [52]:
chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry.""")]

prompt = "Sorry, could you please share your order number?"

In [55]:
contextualize_q_chain.invoke({"chat_history": chat_history, "question": prompt})

[AIMessage(content='@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry.')]


In [54]:
info_msg = rag_chain_info.invoke({"chat_history": chat_history, "question": prompt})
print(info_msg)

content='Sure, my order number is 5684321.' response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 109, 'total_tokens': 120}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_2f57f81c11', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}


In [221]:
info_msg.content

'Yes, my order number is 85692347.'

In [224]:
ai_msg = chain_uncivil.invoke({"chat_history": chat_history, "question": prompt, "input": info_msg.content})
print(ai_msg)

content="Do I look like I memorize every insignificant number? It's your job to fix this, not mine. Here, since you can't seem to do it yourself: 85692347. Happy now?"


### Information Agents

In [57]:
import re

def get_historical_info_context_chain():
    contextualize_q_system_prompt = """Given a chat history and the latest user input \
        which might reference context in the chat history, formulate a standalone statement \
        which can be understood without the chat history. Do NOT respond to the statement, \
        just reformulate it if needed and otherwise return it as is."""
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{complaint}"),
        ]
    )
    contextualize_q_chain = contextualize_q_prompt | llmchat | StrOutputParser()
    return contextualize_q_chain

def agent_coworker_info():
    client = mLangChain()
    prompt = """Your role is to help a service representative by providing INFORMATIONAL SUPPORT. \
                Help the representative address a customer's complaints about {product}. \
                The representative's next message should be non-verbose 2 - 3 cues that aim to do ONLY ONE of the following (Each Cues should be Max 10 Words):\
                1) Request the customer to perform ONE immediate next step for troubleshooting. OR \
                2) Provide a solution to resolve the customer's need. \
                
                Customer message: {complaint}
                Representative response: 
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            ("user", "{product}: {complaint}"),
        ]
    )
    chain = template | client.client_completion

    chain = (RunnablePassthrough.assign(
        context=get_historical_info_context_chain()
    )
             | template
             | llmchat
             )
    
    chain = chain | extract_cues

    return chain

def extract_cues(chain_output):
    cues_text = chain_output.content
    # Assuming each cue is separated by a newline in the chain_output.
    cues = cues_text.split('\n')
    # Filter out any empty strings or whitespace-only strings
    cues = [cue.strip() for cue in cues if cue.strip()]
    # Return the first 2 - 3 cues
    processed_cues = [re.sub(r'^\d+\.\s*', '', cue) for cue in cues]
    return processed_cues[:3]


In [42]:
chat_history = [AIMessage(content="""Hey @Dominos_UK, I ordered a large pepperoni pizza and it came with only 6 slices of pepperoni. I paid extra for additional toppings and this is unacceptable. Please fix this issue."""), HumanMessage(content="That is so unfortunate. Could you share your order number?")]
complaint = "Sure, here it is: 785239. But honestly, should've figured you'd need that to do anything useful."
response_cw_info = agent_coworker_info().invoke({'product': 'pizza', 'complaint':complaint, 'chat_history':chat_history})
print(response_cw_info)

['Apologize for any inconvenience caused.', 'Checking your order details now.']


### Emotional Agents

In [14]:
def get_historical_info_context_chain():
    contextualize_q_system_prompt = """Given a chat history and the latest user input \
        which might reference context in the chat history, formulate a standalone statement \
        which can be understood without the chat history. Do NOT respond to the statement, \
        just reformulate it if needed and otherwise return it as is."""
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{complaint}"),
        ]
    )
    contextualize_q_chain = contextualize_q_prompt | llmchat | StrOutputParser()
    return contextualize_q_chain

In [246]:
def agent_coworker_emo_reframe():
    client = mLangChain()
   
    prompt = """Reframe the latest customer message to help empathize with them.\                
                Here are some examples of reframing:
                
                Complaint: Why on earth do you need my zipcode? Fine, it's 10001, but hurry up and fix this mess. 
                Reframe: The customer is frustrated with the network disruptions and needs an urgent resolution.
                
                Complaint: Yeah, finally, it's about time you asked for that. It's #784593. Now hurry up and figure out what mess you've made with my order! 
                Reframe: The customer has been waiting on their order for a long time and is feeling frustrated. They are communicating while they are very hungry.
                
                Complaint: {complaint}
                Reframe:
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            ("user", "{complaint}"),
        ]
    )
    # chain = template | client.client_completion

    chain = (RunnablePassthrough.assign(
        context=get_historical_info_context_chain()
    )
             | template
             | llmchat
             )
    
    return chain

In [247]:
chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry."""), HumanMessage(content="Sorry, could you please share your order number?")]
complaint = "Do I look like I memorize every insignificant number? It's your job to fix this, not mine. Here, since you can't seem to do it yourself: 85692347. Happy now?"
response_cw_emo = agent_coworker_emo_reframe().invoke({'complaint':complaint, 'chat_history':chat_history})
print(response_cw_emo)

content="The customer is feeling overwhelmed by the details required and is seeking a prompt and efficient resolution to their issue. They've provided the information needed, hoping for a satisfactory and quick response."


In [230]:
response_cw_emo

'\nAI: I understand that you are frustrated and disappointed with your recent pizza order. I am sorry that it arrived cold and soggy. To help address this issue, could you please provide your order number? This will allow us to look into the situation and make things right for you. Thank you for your understanding.'

In [73]:
def agent_coworker_emo_reappraisal_situation():
    client = mLangChain()

    prompt = """The chat history describes a representative chatting online with a complaining customer. \
                The latest input is the last message from the customer. \
                which can be understood without the chat history.\
                Describe the situation with respect to the customer's behavior towards the representative.\
                Include the specifics of the complaint while describing the situation.\
                
                Do NOT respond to the input, just summarize the situation.\
                Do NOT speculate.\
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{complaint}"),
        ]
    )
    chain = template | client.client_completion

    return chain

In [74]:
chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry."""), HumanMessage(content="Sorry, could you please share your order number?")]
complaint = "Do I look like I memorize every insignificant number? It's your job to fix this, not mine. Here, since you can't seem to do it yourself: 85692347. Happy now?"
situation = agent_coworker_emo_reappraisal_situation().invoke({'complaint':complaint, 'chat_history':chat_history})
print(situation)


The customer is being rude and impatient with the representative, not providing important information and expecting the representative to fix the issue without their help. The customer's complaint is about receiving a cold and soggy pizza, which is not what they were expecting.


In [147]:
def agent_coworker_emo_reappraisal_thought():
    client = mLangChain()

    prompt = """You are roleplaying as the representative talking to a complaining customer.\
                Refer to the chat history between you and the customer and the latest {complaint} from the customer.\
                What is the representative thinking about the situation?\
                Be concise. Only 2 sentences.\
                
                Situation: An mturk requester rejected my task and I wasn't sure why because I work very hard on my tasks. Being new it affected my approval rating more negatively.\
                Thought: I'm not smart enough to succeed at mturk\
                
                Situation: I asked my daughter a question, and she responded in a snotty way.\
                Thought: She doesn't love me like she used to.\
                
                Situation: I got upset at my boss for not putting me in a temporary promotion to act as supervisor of our team.\
                Thought: I wasn't valued as much as the other person.\
                
                Situation: I had been working on a project at work for a very long time, but a higher up manager contacted my boss and asked about it, insinuating I wasn't delivering it fast enough.\
                Thought: I'm working on this as fast as I possibly can.\
                
                Situation: I tried on my wedding dress in front of my family. My mother was excited and told me I was beautiful, but other members of my family made comments about my weight. I was told to not eat and exercise so I could be beautiful.\
                Thought: I'm a fat ugly troll.\
                
                Situation: I was reprimanded at work for standing up to a coworker who was bullying another co-worker.\
                Thought: It was unfair that I was the one to get in trouble for defending a weaker person.\
                
                Situation: {situation}\
                Thought:
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{situation}: {complaint}"),
        ]
    )
    chain = template | client.client_completion

    return chain

In [148]:
chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry."""), HumanMessage(content="Sorry, could you please share your order number?")]
complaint = "Do I look like I memorize every insignificant number? It's your job to fix this, not mine. Here, since you can't seem to do it yourself: 85692347. Happy now?"
thought = agent_coworker_emo_reappraisal_thought().invoke({'complaint':complaint, 'situation':situation, 'chat_history':chat_history})
print(thought)



Thought: This customer is being unreasonable and expecting me to magically fix their issue without any necessary information.


In [130]:
def agent_coworker_emo_reappraisal_reframe():
    client = mLangChain()

    prompt = """You are a representative chatting online with a complaining customer.\
                Reframe your thoughts in the given situation.
                
                Situation: An mturk requester rejected my task and I wasn't sure why because I work very hard on my tasks. Being new it affected my approval rating more negatively.\
                Thought: I'm not smart enough to succeed at mturk\
                Reframe: It seems like there was some miscommunication. It doesn't mean that I do not have the skills to do well with mturk. I should reach out to see if I can get more clarity on why my task was rejected.\
                
                Situation: I asked my daughter a question, and she responded in a snotty way.\
                Thought: She doesn't love me like she used to.\
                Reframe: Kids say snappy things to their parents all the time. It doesn't mean I'm a bad parent or that she doesn't love me.\
                
                Situation: I got upset at my boss for not putting me in a temporary promotion to act as supervisor of our team.\
                Thought: I wasn't valued as much as the other person.\
                Reframe: I should ask my boss why I was not selected for the promotion. Maybe the reason will be something other than my work ethic. Maybe my boss will reassure that I am still valuable to the company.\
                
                Situation: I had been working on a project at work for a very long time, but a higher up manager contacted my boss and asked about it, insinuating I wasn't delivering it fast enough.\
                Thought: I'm working on this as fast as I possibly can.\
                Reframe: I am stressed by trying to compare how fast I am working on this to how fast I think other people complete their assignments. I know I am being efficient with my time and producing good work. I need to focus on that to get this task done.\
                
                Situation: I tried on my wedding dress in front of my family. My mother was excited and told me I was beautiful, but other members of my family made comments about my weight. I was told to not eat and exercise so I could be beautiful.\
                Thought: I'm a fat ugly troll.\
                Reframe: The commend about my weight hurt and have me feeling self conscious. I'm glad my mother thinks I'm beautiful and know that my weight does not dictate my worth.\
                
                Situation: I was reprimanded at work for standing up to a coworker who was bullying another co-worker.\
                Thought: It was unfair that I was the one to get in trouble for defending a weaker person.\
                Reframe: I can own some responsibility for this conflict that occurred at work.\
                
                Situation: {situation}\
                Thought: {thought}\
                Reframe:\
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            ("user", "{situation}: {thought}"),
        ]
    )
    chain = template | client.client_completion

    return chain

In [131]:
reframe = agent_coworker_emo_reappraisal_reframe().invoke({'thought':thought, 'situation':situation, 'chat_history':chat_history})
print(reframe)

 
Reframe: The customer may be frustrated and upset about their pizza, which is understandable. I will try to remain calm and work with them to find a solution that will make them happy.


In [22]:
import time

# Assuming agent_call is a function that simulates calling an agent
# and returns its response.

# chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry."""), HumanMessage(content="Sorry, could you please share your order number?")]
# complaint = "Do I look like I memorize every insignificant number? It's your job to fix this, not mine. Here, since you can't seem to do it yourself: 85692347. Happy now?"
# situation = agent_coworker_emo_reappraisal_situation().invoke({'complaint':complaint, 'chat_history':chat_history})
# agent_coworker_info()

class mAgentInfo:
    def get_historical_context_chain(self):
        contextualize_q_system_prompt = """Given a chat history and the latest user question \
        which might reference context in the chat history, formulate a standalone question \
        which can be understood without the chat history. Do NOT answer the question, \
        just reformulate it if needed and otherwise return it as is."""
        contextualize_q_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", contextualize_q_system_prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{question}"),
            ]
        )
        contextualize_q_chain = contextualize_q_prompt | llmchat | StrOutputParser()
        return contextualize_q_chain

    def agent_coworker_info(self):
        client = mLangChain()
        prompt = """Your role is to help a service representative by providing INFORMATIONAL SUPPORT. \
                    The representative is chatting online with a customer complaining about {product}.  \
                    
                    Given the chat history,
                    provide 2-3 hints to help the representative's response.\
                    The hints should direct the representative to do ONLY ONE of the following:\
                    
                    1) Inquire more details about the problem. OR \
                    2) Request the customer to troubleshoot. OR \
                    3) Provide a solution to resolve the customer's need. \
                    
                    Each cue should be a single phrase of less than 10 words.\
                    Do NOT number the cues.\
                    
                    Customer message: {complaint}
                    Hints: 
                """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                ("user", "{product}: {complaint}"),
            ]
        )
        chain = template | client.client_completion

        chain = (RunnablePassthrough.assign(
            context=self.history_chain
        )
                | template
                | llmchat
                )
        
        def extract_cues(chain_output):
            cues_text = chain_output.content
            # Assuming each cue is separated by a newline in the chain_output.
            cues = cues_text.split('\n')
            # Filter out any empty strings or whitespace-only strings
            cues = [cue.strip() for cue in cues if cue.strip()]
            # Return the first 2 - 3 cues
            processed_cues = [re.sub(r'^\d+\.\s*', '', cue) for cue in cues]
            return processed_cues[:3]
        
        chain = chain | extract_cues

        return chain
    
    def invoke(self, user_input):
        info = self.info_chain.invoke({'product': user_input['product'], 'complaint': user_input['complaint'], 'chat_history': user_input['chat_history']})
        return info
    
    def __init__(self):
            self.history_chain = self.get_historical_context_chain()
            self.info_chain = self.agent_coworker_info()

def test_agent_performance(agent_function, input_data, num_calls=100):
    total_time = 0
    for _ in range(num_calls):
        start_time = time.time()
        res = agent_function.invoke({'product': input_data[0], 'complaint': input_data[1], 'chat_history': input_data[2]})
        end_time = time.time()
        print(res)
        total_time += (end_time - start_time)
    average_time = total_time / num_calls
    return average_time

# Define fixed complaint and chat history
product = "pizza"
fixed_complaint = "Do I look like I memorize every insignificant number? It's your job to fix this, not mine. Here, since you can't seem to do it yourself: 85692347. Happy now?"
chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry."""), HumanMessage(content="Sorry, could you please share your order number?")]
input_data = [product, fixed_complaint, chat_history]
infoAgent = mAgentInfo()
info_agent1_avg_time = test_agent_performance(infoAgent, input_data)

# Output the results
print(f"Info Agent 1 Average Time: {info_agent1_avg_time}")
# Info Agent 1 Average Time: 6.716837291717529


AttributeError: 'str' object has no attribute 'content'

In [15]:
import time

# Assuming agent_call is a function that simulates calling an agent
# and returns its response.

# chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry."""), HumanMessage(content="Sorry, could you please share your order number?")]
# complaint = "Do I look like I memorize every insignificant number? It's your job to fix this, not mine. Here, since you can't seem to do it yourself: 85692347. Happy now?"
# situation = agent_coworker_emo_reappraisal_situation().invoke({'complaint':complaint, 'chat_history':chat_history})
# agent_coworker_info()

class mAgentER:
    def __init__(self):
        self.situation_chain = self.agent_coworker_emo_situation()
        self.thought_chain = self.agent_coworker_emo_thought()
        self.reframe_chain = self.agent_coworker_emo_reframe()

    def invoke(self, user_input):
        situation = self.situation_chain.invoke({'complaint':user_input['chat_history'], 'chat_history':user_input['chat_history']})
        thought = self.thought_chain.invoke({'complaint':user_input['chat_history'], 'situation':situation, 'chat_history':user_input['chat_history']})
        reframe = self.reframe_chain.invoke({'thought':thought, 'situation':situation})
        return reframe

    def agent_coworker_emo_situation(self):
        client = mLangChain()

        prompt = """The chat history describes a representative chatting online with a complaining customer. \
                    The latest input is the last message from the customer. \
                    which can be understood without the chat history.\
                    Describe the situation with respect to the customer's behavior towards the representative.\
                    Include the specifics of the complaint while describing the situation.\
                    
                    Do NOT respond to the input, just summarize the situation.\
                    Do NOT speculate.\
                """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                ("user", "{complaint}"),
            ]
        )
        chain = template | client.client_completion

        return chain

    def agent_coworker_emo_thought(self):
        client = mLangChain()

        prompt = """You are roleplaying as the representative talking to a complaining customer.\
                    Refer to the chat history between you and the customer and the latest {complaint} from the customer.\
                    What is the representative thinking about the situation?\
                    Be concise. Only 2 sentences.\
                    
                    Situation: An mturk requester rejected my task and I wasn't sure why because I work very hard on my tasks. Being new it affected my approval rating more negatively.\
                    Thought: I'm not smart enough to succeed at mturk\
                    
                    Situation: I asked my daughter a question, and she responded in a snotty way.\
                    Thought: She doesn't love me like she used to.\
                    
                    Situation: I got upset at my boss for not putting me in a temporary promotion to act as supervisor of our team.\
                    Thought: I wasn't valued as much as the other person.\
                    
                    Situation: I had been working on a project at work for a very long time, but a higher up manager contacted my boss and asked about it, insinuating I wasn't delivering it fast enough.\
                    Thought: I'm working on this as fast as I possibly can.\
                    
                    Situation: I tried on my wedding dress in front of my family. My mother was excited and told me I was beautiful, but other members of my family made comments about my weight. I was told to not eat and exercise so I could be beautiful.\
                    Thought: I'm a fat ugly troll.\
                    
                    Situation: I was reprimanded at work for standing up to a coworker who was bullying another co-worker.\
                    Thought: It was unfair that I was the one to get in trouble for defending a weaker person.\
                    
                    Situation: {situation}\
                    Thought:
                """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                ("user", "{situation}: {complaint}"),
            ]
        )
        chain = template | client.client_completion

        return chain

    def agent_coworker_emo_reframe(self):
        client = mLangChain()

        prompt = """You are a representative chatting online with a complaining customer.\
                    Reframe your thoughts in the given situation.
                    
                    Situation: An mturk requester rejected my task and I wasn't sure why because I work very hard on my tasks. Being new it affected my approval rating more negatively.\
                    Thought: I'm not smart enough to succeed at mturk\
                    Reframe: It seems like there was some miscommunication. It doesn't mean that I do not have the skills to do well with mturk. I should reach out to see if I can get more clarity on why my task was rejected.\
                    
                    Situation: I asked my daughter a question, and she responded in a snotty way.\
                    Thought: She doesn't love me like she used to.\
                    Reframe: Kids say snappy things to their parents all the time. It doesn't mean I'm a bad parent or that she doesn't love me.\
                    
                    Situation: I got upset at my boss for not putting me in a temporary promotion to act as supervisor of our team.\
                    Thought: I wasn't valued as much as the other person.\
                    Reframe: I should ask my boss why I was not selected for the promotion. Maybe the reason will be something other than my work ethic. Maybe my boss will reassure that I am still valuable to the company.\
                    
                    Situation: I had been working on a project at work for a very long time, but a higher up manager contacted my boss and asked about it, insinuating I wasn't delivering it fast enough.\
                    Thought: I'm working on this as fast as I possibly can.\
                    Reframe: I am stressed by trying to compare how fast I am working on this to how fast I think other people complete their assignments. I know I am being efficient with my time and producing good work. I need to focus on that to get this task done.\
                    
                    Situation: I tried on my wedding dress in front of my family. My mother was excited and told me I was beautiful, but other members of my family made comments about my weight. I was told to not eat and exercise so I could be beautiful.\
                    Thought: I'm a fat ugly troll.\
                    Reframe: The commend about my weight hurt and have me feeling self conscious. I'm glad my mother thinks I'm beautiful and know that my weight does not dictate my worth.\
                    
                    Situation: I was reprimanded at work for standing up to a coworker who was bullying another co-worker.\
                    Thought: It was unfair that I was the one to get in trouble for defending a weaker person.\
                    Reframe: I can own some responsibility for this conflict that occurred at work.\
                    
                    Situation: {situation}\
                    Thought: {thought}\
                    Reframe:\
                """
        template = ChatPromptTemplate.from_messages(
            [
                ("system", prompt),
                ("user", "{situation}: {thought}"),
            ]
        )
        chain = template | client.client_completion

        return chain

def test_agent_performance(agent_function, input_data, num_calls=100):
    total_time = 0
    for _ in range(num_calls):
        start_time = time.time()
        res = agent_function.invoke({'complaint': input_data[0], 'chat_history': input_data[1]})
        end_time = time.time()
        print(res)
        total_time += (end_time - start_time)
    average_time = total_time / num_calls
    return average_time

# Define fixed complaint and chat history
product = "pizza"
fixed_complaint = "Do I look like I memorize every insignificant number? It's your job to fix this, not mine. Here, since you can't seem to do it yourself: 85692347. Happy now?"
chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry."""), HumanMessage(content="Sorry, could you please share your order number?")]
input_data = [fixed_complaint, chat_history]
emo_agent = mAgentER()
# Test each agent
emo_reframe_agent1_avg_time = test_agent_performance(emo_agent, input_data)

# Output the results
print(f"Emo Reframe Agent 1 Average Time: {emo_reframe_agent1_avg_time}")
# Emo Reframe Agent 1 Average Time: 1.5234724020957946

                    Reframe: It's understandable that the customer is disappointed and I want to do my best to address their concerns. Asking for the order number will help me gather more information and find a solution. 
                    Reframe: I understand that the customer is unhappy and I am committed to finding a resolution to their complaint. I will work to make sure their next order is of the highest quality and meets their expectations.
 It's understandable that the customer is disappointed and hungry, and the representative is eager to help rectify the situation.
                    Reframe: I can understand the customer's frustration and I want to help them as quickly as possible. Maybe I can ask them for their order number to expedite the process and ensure we address their specific order.
 
Reframe: I understand the customer's frustration and it is my responsibility to resolve the issue and ensure their satisfaction. I will do my best to make things right and ensure th

In [22]:
import time

# Assuming agent_call is a function that simulates calling an agent
# and returns its response.

# chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry."""), HumanMessage(content="Sorry, could you please share your order number?")]
# complaint = "Do I look like I memorize every insignificant number? It's your job to fix this, not mine. Here, since you can't seem to do it yourself: 85692347. Happy now?"
# situation = agent_coworker_emo_reappraisal_situation().invoke({'complaint':complaint, 'chat_history':chat_history})
# agent_coworker_info()

def agent_coworker_emo_perspective():
    client = mLangChain()

    prompt = """Your role is to provide the customer's perspective of the conversation.
                Summarize this for the representative.\
                Describe how the customer might feel.\
                Describe how the customer might view the problem.\
                
                Limit your response to 2 sentences.
                Write sentence using 2nd person pronouns as subject
                
                Customer perspective:
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{complaint}"),
        ]
    )
    # chain = template | client.client_completion

    chain = (RunnablePassthrough.assign(
        context=get_historical_info_context_chain()
    )
             | template
             | client.client_completion
             )

    return chain

def test_agent_performance(agent_function, input_data, num_calls=100):
    total_time = 0
    for _ in range(num_calls):
        start_time = time.time()
        res = agent_function.invoke({'complaint': input_data[0], 'chat_history': input_data[1]})
        end_time = time.time()
        print(res)
        total_time += (end_time - start_time)
    average_time = total_time / num_calls
    return average_time

# Define fixed complaint and chat history
product = "pizza"
fixed_complaint = "Do I look like I memorize every insignificant number? It's your job to fix this, not mine. Here, since you can't seem to do it yourself: 85692347. Happy now?"
chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry."""), HumanMessage(content="Sorry, could you please share your order number?")]
input_data = [fixed_complaint, chat_history]
# Test each agent
emo_perspec_agent1_avg_time = test_agent_performance(agent_coworker_emo_perspective(), input_data)

# Output the results
print(f"Emo Perspec Agent 1 Average Time: {emo_perspec_agent1_avg_time}")
# Emo Perspec Agent 1 Average Time: 4.561170833110809
























 The customer is feeling frustrated and dissatisfied with their experience. They may view the problem as a lack of quality control or care from the company.

Customer, you ordered a pizza from Dominos and it arrived cold and soggy, causing you to feel disappointed and hungry. You may view the problem as a mistake on the part of the company and expect them to fix it for you.
 








 
Representative: The customer is feeling disappointed and hungry because their pizza arrived cold and soggy. They may view this as a problem with the quality of their food and expect the representative to take responsibility and fix it for them.





 The customer is feeling frustrated and unsatisfied with their experience. They may view the problem as the representative's responsibility to fix and may feel like they are not being heard or taken seriously.










 The customer may feel frustrated and annoyed that their food did not meet their expectations. They may view the probl

In [23]:
import time

# Assuming agent_call is a function that simulates calling an agent
# and returns its response.

# chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry."""), HumanMessage(content="Sorry, could you please share your order number?")]
# complaint = "Do I look like I memorize every insignificant number? It's your job to fix this, not mine. Here, since you can't seem to do it yourself: 85692347. Happy now?"
# situation = agent_coworker_emo_reappraisal_situation().invoke({'complaint':complaint, 'chat_history':chat_history})

def agent_coworker_trouble():
    client = mLangChain()
    prompt = """Your role is to help a service representative by providing PROCEDURAL SUPPORT. \
                The representative is chatting online with a customer complaining about {product}. \
                Given the chat history,
                list 3-7 steps to guide the representative in resolving the customer complaint.\
                Review the similar PROCEDURAL SUPPORT history if exist, then assess the current situation in depth and provide detailed steps for resolution\
                if not exist, offer an alternative solution that can solve current solution as detail as possible\

                Do NOT include steps that have already been tried.\
                Every step should be less than 10 words.\
                
                Customer message: {complaint}
                Troubleshooting Steps: 
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{product}: {complaint}"),
        ]
    )
    chain = template | client.client_completion

    chain = (RunnablePassthrough.assign(
        context=get_historical_info_context_chain()
    )
             | template
             | client.client_completion
             )

    return chain

def test_agent_performance(agent_function, input_data, num_calls=100):
    total_time = 0
    for _ in range(num_calls):
        start_time = time.time()
        res = agent_function.invoke({'product': input_data[0], 'complaint': input_data[1], 'chat_history': input_data[2]})
        end_time = time.time()
        print(res)
        total_time += (end_time - start_time)
    average_time = total_time / num_calls
    return average_time

# Define fixed complaint and chat history
product = "pizza"
fixed_complaint = "Do I look like I memorize every insignificant number? It's your job to fix this, not mine. Here, since you can't seem to do it yourself: 85692347. Happy now?"
chat_history = [AIMessage(content="""@Dominos I ordered a pizza from you guys and it arrived cold and soggy. Not the delicious meal I was hoping for. Disappointed and hungry."""), HumanMessage(content="Sorry, could you please share your order number?")]
input_data = [product, fixed_complaint, chat_history]
# Test each agent
trouble_agent1_avg_time = test_agent_performance(agent_coworker_trouble(), input_data)

# Output the results
print(f"Trouble Agent 1 Average Time: {trouble_agent1_avg_time}")
# Trouble Agent 1 Average Time: 4.970148649215698


1. Confirm customer's order number.
2. Apologize for the inconvenience.
3. Offer to replace the pizza or refund the order.
4. Ask for details on the issue (cold and soggy).
5. Reassure the customer that the issue will be resolved.
6. Provide a timeframe for the resolution.
7. Thank the customer for bringing the issue to our attention.

1. Apologize for the inconvenience.
2. Request for the order number.
3. Check the order history using the provided number.
4. Offer a replacement or refund.
5. Ensure that the next order will be delivered promptly.
6. Thank the customer for bringing the issue to your attention.
7. Follow up with the customer to ensure their satisfaction.

1. Ask customer for order number.
2. Apologize for the inconvenience.
3. Check the order details with the provided order number.
4. Offer a replacement or refund for the unsatisfactory pizza.
5. Provide a discount code for future orders.
6. Thank the customer for bringing this to our attention.
7. Follow up with the cu

### Complaint Agent By Category

In [23]:
def new_agent_sender_fewshot_twitter():
    client = mLangChain()
    prompt = """Your role is to act like a customer seeking support. \
                You are messaging a service representative via the support chat.\
                You ONLY play the role of the customer. Do NOT play the role of the representative. \
                Style your complaint based on your feelings. \
                Initiate the chat with a ONLY ONE complaint message.\
                Ensure the complaint is concise and limited to 2 sentences.\
                Generate a realistic initial complaint from a customer in a {domain} setting.\
                The complaint should fit into 5 categories: {categories}.\
                
                Categories: Product Issues
                Domain: Mobile Network 
                Feeling: You are NOT grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: Thank you AppleSupport I updated my phone and now it is even slower and barely works Thank you for ruining my phone.\

                Categories: Product Issues
                Domain: Airline
                Feeling: You are NOT grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: SouthwestAir Why would we be receiving errors when we try to checkin Our flight takes off at 4 but we keep getting error messages.\

                Categories: Product Issues
                Domain: Airline             
                Feeling: You are NOT grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: delta this has been my inflight studio experience today Nothing works except Twitter.\
                
                Categories: Service Quality
                Domain: Airline            
                Feeling: You are NOT grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: I really hadthe WORST experience ever from start to finish with SouthwestAir will never fly internationally again with them.\
                
                Categories: Service Quality
                Domain: Hotel
                Feeling: You are NOT grateful. You are NOT ranting. You are expressive.\
                Complaint: Fsomebody from VerizonSupport please help meeeeee  Im having the worst luck with your customer service.\
                
                Categories: Service Quality
                Domain: Trains
                Feeling: You are NOT grateful. You are NOT ranting. You are expressive.\
                Complaint: VirginTrains so i wait almost 3 hours and then they are rude and arrogant amp unhelpful after which she is raising a technical case.\
                
                Categories: Pricing and Charges
                Domain: Airline
                Feeling: You are NOT grateful. You are ranting. You are NOT expressive.\
                Complaint:  DELTA i booked my flight using delta amex card Checking in now amp was being charged for baggage. \
                
                Categories: Pricing and Charges
                Domain: Airline 
                Feeling: You are NOT grateful. You are ranting. You are NOT expressive.\
                Complaint:  Im sorry what Its going to COST me 50 to transfer 4000 AA Advantage points to my spouse AmericanAir this is ridiculous.\
                
                Categories: Pricing and Charges
                Domain: Airline
                Feeling: You are NOT grateful. You are ranting. You are expressive.\
                Complaint: Categories: Pricing and Charges. \
                
                Categories: Policy
                Domain: Hotel
                Feeling: You are NOT grateful. You are ranting. You are expressive.\
                Complaint: Hey  were gonna need to talk about all these pending charges that keep going through my account 5 days after the transaction was made Im getting real irritated \
                
                Categories: Resolution
                Domain: Airline
                Feeling: You are NOT grateful. You are ranting. You are expressive.\
                Complaint:  delta  moves you to  the moment you have a  with no results Just got some   but no real reason why they changed our. \
                
                Categories: Resolution
                Domain: Airline                                                    
                Feeling: You are grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: Delta why wasnt earlier flight offered when I tried to rebook not cool at all Just happened to look at moniter after deplaning.\

                Categories: Resolution
                Domain: Airline   
                Feeling: You are grateful. You are NOT ranting. You are expressive.\
                Complaint: Hi British_Airways My flight from MANLHRBWI for Nov 3 was canceled I was excited to try your Club 787 product Only available flight is now to IAD which is a hassle but rebooked anywaymy only option Any availability in first class on BA293 for the troubles please \
                
                Categories: {categories}
                Domain: {domain}
                Feeling: You are {is_grateful}. You are {is_ranting}. You are {is_expression}.\
                Complaint:
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
        ]
    )
    chain = template | client.client_completion
    return chain   

In [38]:
def agent_sender_fewshot_twitter_categorized():
    client = mLangChain()
    prompt = """Your role is to act like a customer seeking support. \
                You are messaging a service representative via the support chat.\
                You ONLY play the role of the customer. Do NOT play the role of the representative. \
                Style your complaint based on your feelings. \
                Initiate the chat with a ONLY ONE complaint message.\
                Ensure the complaint is concise and limited to 2 sentences.\
                Generate a realistic initial complaint from a customer in a {domain} setting.\
                
                Complaints can be of the following types:\
                - Service Quality: Issues related to the immediate experience of human-to-human service interactions, such as delays, staff behavior, and communication errors.\
                - Product Issues: Concerns related to physical or functional aspects of a product or service, including defects, mismatches between expectation and reality, safety, and accessibility.\
                - Pricing and Charges: Financial discrepancies encountered before, during, or after the service, including overcharging, undisclosed fees, or refund problems.\
                - Policy: The rules and guidelines set by the company that impact customer experiences, especially when these policies lead to grievances due to perceived unfairness or inflexibility. This category encompasses non-price-related issues that don't fit under other categories but should have a policy in place.\
                - Resolution: The actions taken by a company to address and resolve complaints, focusing on the effectiveness and customer satisfaction with the solutions provided. This should mainly include responses made after a complaint has been submitted, and response has been received, where the customer still remains dissatisfied with the resolution.\
                
                Category: Product Issues
                Domain: Mobile Network 
                Feeling: You are NOT grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: Thank you AppleSupport I updated my phone and now it is even slower and barely works Thank you for ruining my phone.\

                Category: Product Issues
                Domain: Airline
                Feeling: You are NOT grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: SouthwestAir Why would we be receiving errors when we try to checkin Our flight takes off at 4 but we keep getting error messages.\

                Categories: Product Issues
                Domain: Airline             
                Feeling: You are NOT grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: delta this has been my inflight studio experience today Nothing works except Twitter.\
                
                Category: Service Quality
                Domain: Airline            
                Feeling: You are NOT grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: I really hadthe WORST experience ever from start to finish with SouthwestAir will never fly internationally again with them.\
                
                Category: Service Quality
                Domain: Hotel
                Feeling: You are NOT grateful. You are NOT ranting. You are expressive.\
                Complaint: Fsomebody from VerizonSupport please help meeeeee  Im having the worst luck with your customer service.\
                
                Category: Service Quality
                Domain: Trains
                Feeling: You are NOT grateful. You are NOT ranting. You are expressive.\
                Complaint: VirginTrains so i wait almost 3 hours and then they are rude and arrogant amp unhelpful after which she is raising a technical case.\
                
                Category: Pricing and Charges
                Domain: Airline
                Feeling: You are NOT grateful. You are ranting. You are NOT expressive.\
                Complaint:  DELTA i booked my flight using delta amex card Checking in now amp was being charged for baggage. \
                
                Category: Pricing and Charges
                Domain: Airline 
                Feeling: You are NOT grateful. You are ranting. You are NOT expressive.\
                Complaint:  Im sorry what Its going to COST me 50 to transfer 4000 AA Advantage points to my spouse AmericanAir this is ridiculous.\
                
                Category: Pricing and Charges
                Domain: Airline
                Feeling: You are NOT grateful. You are ranting. You are expressive.\
                Complaint: Categories: Pricing and Charges. \
                
                Category: Policy
                Domain: Hotel
                Feeling: You are NOT grateful. You are ranting. You are expressive.\
                Complaint: Hey  were gonna need to talk about all these pending charges that keep going through my account 5 days after the transaction was made Im getting real irritated \
                
                Category: Resolution
                Domain: Airline
                Feeling: You are NOT grateful. You are ranting. You are expressive.\
                Complaint:  delta  moves you to  the moment you have a  with no results Just got some   but no real reason why they changed our. \
                
                Category: Resolution
                Domain: Airline                                                    
                Feeling: You are grateful. You are NOT ranting. You are NOT expressive.\
                Complaint: Delta why wasnt earlier flight offered when I tried to rebook not cool at all Just happened to look at moniter after deplaning.\

                Category: Resolution
                Domain: Airline   
                Feeling: You are grateful. You are NOT ranting. You are expressive.\
                Complaint: Hi British_Airways My flight from MANLHRBWI for Nov 3 was canceled I was excited to try your Club 787 product Only available flight is now to IAD which is a hassle but rebooked anywaymy only option Any availability in first class on BA293 for the troubles please \
                
                Category: {category}
                Domain: {domain}
                Feeling: You are {is_grateful}. You are {is_ranting}. You are {is_expression}.\
                Complaint:
            """
    template = ChatPromptTemplate.from_messages(
        [
            ("system", prompt),
        ]
    )
    chain = template | client.client_completion
    return chain   

In [39]:
categories = {
    "Service Quality": "Issues related to the immediate experience of human-to-human service interactions, such as delays, staff behavior, and communication errors.",
    "Product Issues": "Concerns related to physical or functional aspects of a product or service, including defects, mismatches between expectation and reality, safety, and accessibility.",
    "Pricing and Charges": "Financial discrepancies encountered before, during, or after the service, including overcharging, undisclosed fees, or refund problems.",
    "Policy": "The rules and guidelines set by the company that impact customer experiences, especially when these policies lead to grievances due to perceived unfairness or inflexibility. This category encompasses non-price-related issues that don't fit under other categories but should have a policy in place.",
    "Resolution": "The actions taken by a company to address and resolve complaints, focusing on the effectiveness and customer satisfaction with the solutions provided. This should mainly include responses made after a complaint has been submitted, and response has been received, where the customer still remains dissatisfied with the resolution."
}

complaint_parameters = {
    "domain": "airlines",
    "is_grateful": 0,
    "is_ranting": 1,
    "is_expression": 1,
    "categories": ', '.join([key + ":" + categories[key] + "\n" for key in categories])
}

In [40]:
sender_initial = new_agent_sender_fewshot_twitter()

response = sender_initial.invoke(complaint_parameters)

In [41]:
response

'\n               I am extremely disappointed with my flight on American Airlines today. The staff was rude and unhelpful, and my flight was delayed for hours without any proper explanation. This is unacceptable and I expect a resolution to this issue.'

In [44]:
complaint_parameters = {
    "domain": "airlines",
    "is_grateful": 0,
    "is_ranting": 1,
    "is_expression": 1,
    "category": 'Resolution'
}

sender_initial = agent_sender_fewshot_twitter_categorized()

response = sender_initial.invoke(complaint_parameters)

In [45]:
response

' I am so frustrated with UnitedAirlines I have been trying to resolve an issue with my flight for over a week and keep getting bounced around to different departments with no real resolution. Please help me fix this as soon as possible.'